In [ ]:
import community
import networkx as nx
import numpy as np
from  gensim import models
from scipy.sparse import *
from collections import defaultdict


base_relevant=3
#base_relevant=-1

class Louvain(object):
    def __init__(self, id2pw: dict):
        self.id2pw = id2pw
        self.rbcids = []
        self.cluster_name = "louvain"
    def save_cluster(self):
        file = open(self.cluster_name, 'w', encoding='utf8')
        for item in self.rbcids:
            for i in range(len(item)):
                pw = item[i]
                try:
                    cid1=pw.split('-')[0]
                    cid1_name=id1_name[cid1]
                    pw=cid1_name+'-'+pw.split('-')[1]
                    file.write(pw+", " if i < len(item)-1 else pw)
                except:
                    print(pw)
            file.write("\n")
        file.close()
    def run_from_graph(self, G):
        partition = community.best_partition(G,)
        size = float(len(set(partition.values())))
        pos = nx.spring_layout(G)
        count = 0.
        for com in set(partition.values()):
            count = count + 1.
            list_nodes = [nodes for nodes in partition.keys()
                          if partition[nodes] == com]
            print(list_nodes)
            self.rbcids.append(list_nodes)
            nx.draw_networkx_nodes(G, pos, list_nodes, node_size=20,
                                   node_color=str(count / size))
        nx.draw_networkx_edges(G, pos, alpha=0.5)
        plt.show()

    def run_from_mat(self, sim_mat):
        x, y = sim_mat.shape
        print(x,y)
        G = nx.Graph()
        for key1 in range(1, x):
            if key1%100==0:
                print(key1)
            if key1+1 < y:
                for key2 in range(key1+1, y):
                    pw1 = self.id2pw.get(key1)
                    pw2 = self.id2pw.get(key2)
                    val = sim_mat[key1, key2]
                    if pw1 is not None and pw2 is not None:
                        G.add_weighted_edges_from([(pw1, pw2, val)])
        self.run_from_graph(G)
def load_Glove_data():
    item={}
    with open('../Glove/vec_glv.txt', encoding='utf8') as file:
        for line in file:
            try:
                vec=[]
                pws=[pw_idx for pw_idx in line.strip().split(" ")]
                id=int(pws[0])
                for content in pws[1:]:
                    vec.append(float(content))
                item[id]=np.array(vec).T
            except:
                continue
    return item 
def load_id1_name():
    id1_name={}
    for line in open('../cid1_name_code.csv', encoding='utf8'):
        cons = line.strip().split("\t")
        try:
            id1_name[cons[0]]=cons[1]
        except:
            print(cons[0])
    return id1_name
def load_id2pw(limit_code):
    id2pw={}
    #用来记录矩阵的行列数
    global all_line
    all_line=0
    i=1
    for line in open('../id2pw.txt', encoding='utf8'):
        all_line+=1
        cons = line.strip().split("\t")
        if i not in limit_code:
            try:
                a=vec_Glove[i]
                id2pw[i]=cons[0]
            except:
                i+=1
                continue
        i+=1
    return id2pw
def limit_frequency():
    #删除低于10个用户点击的商品code
    pw_Min10=[]
    with open('../data.txt', encoding='utf8') as file:
        for line in file:       
            pws=list(pw_idx for pw_idx in line.strip().split("\t"))
            pw=pws[0]
            num=int(pws[1])
            if num<10:
                pw_Min10.append(pw)
    code_Min10=[]
    with open('../id2pw_name.txt', encoding='utf8') as file:
        for line in file:       
            pws=list(pw_idx for pw_idx in line.strip().split("\t"))
            pw=pws[0]
            code=int(pws[1])
            if pw in pw_Min10:
                code_Min10.append(code)
    return code_Min10
def load_data_relevant(filename):
    edges={}
    with open(filename, encoding='utf8') as file:
        for line in file:
            item=line.split(":")
            key=int(item[0])
            cons = [pw for pw in item[1].strip().split(",") if pw!='']
            edges[key]=cons
    return edges
def _func(edges):
    print(len(id2pw))
    sim_mat = lil_matrix(( all_line+1, all_line + 1))
    for i in id2pw.keys():
        lst = edges.get(i)
        if lst is not None:
            for j in lst:
                j=int(j)
                try:
                    vec1 = vec_Glove[i]
                    vec2 = vec_Glove[j]
                    val1 = np.sqrt(sum([item ** 2 for item in vec1]))
                    val2 = np.sqrt(sum([item ** 2 for item in vec2]))
                    sim = vec1.dot(vec2.T) / (val1 * val2)
                    #print(sim)
                    if sim > 0.8:
                        sim_mat[i, j] = sim
                        sim_mat[j, i] = sim
                except Exception as e:
                    #print(e)
                    continue
    return sim_mat
vec_Glove=load_Glove_data()
print("模型加载完毕")
code_Min10=limit_frequency()
print("限制的商品code加载完毕")
id2pw=load_id2pw(code_Min10)
id1_name=load_id1_name()
edges=load_data_relevant('../base_relevant_'+str(base_relevant))
print("相关数据处理完毕，限制最大相关数") 
sim_mat=_func(edges)
A=Louvain(id2pw)
A.run_from_mat(sim_mat)

模型加载完毕
限制的商品code加载完毕
相关数据处理完毕，限制最大相关数
303130
387429 387429
100
200
